# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd
import geoviews as gv
import holoviews as hv
from geoviews import opts
from bokeh.models import WMTSTileSource, ColumnDataSource, DataTable, TableColumn

# Import API key
from api_keys import geoapify_key


In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kingston,17.9970,-76.7936,31.53,70,75,2.57,JM,1722542332
1,1,port alfred,-33.5906,26.8910,11.77,89,17,0.77,ZA,1722542332
2,2,mitsamiouli,-11.3847,43.2844,25.43,67,2,2.81,KM,1722542332
3,3,richards bay,-28.7830,32.0377,19.01,80,6,5.47,ZA,1722542333
4,4,severinia,-20.8094,-48.8028,29.46,20,0,3.98,BR,1722542333


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display
map_data = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM',
                               size='Humidity', color='City', alpha=0.6, scale=1, frame_height=600, frame_width=800)
map_data

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness
# YOUR CODE HERE
filtered_df = city_data_df[(city_data_df['Max Temp'] < 30 ) & 
                           (city_data_df['Wind Speed'] < 4.5) & 
                           (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df.head(17)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,severinia,-20.8094,-48.8028,29.46,20,0,3.98,BR,1722542333
20,20,la passe,45.5549,-0.8967,24.13,82,0,3.94,FR,1722542337
27,27,saint-francois,46.4154,3.9054,24.69,76,0,4.12,FR,1722542338
51,51,udachny,66.4167,112.4000,7.49,65,0,1.43,RU,1722542344
73,73,longyearbyen,78.2186,15.6401,12.91,67,0,1.54,SJ,1722542351
117,117,arbatache,36.6377,3.3713,29.18,66,0,2.06,DZ,1722542362
119,119,katsuren-haebaru,26.3369,127.8719,29.21,86,0,3.09,JP,1722542362
137,137,mpongwe,-13.5091,28.1550,17.56,32,0,3.05,ZM,1722542366
174,174,emerald,-23.5333,148.1667,5.75,55,0,3.54,AU,1722542376
182,182,kodiak,57.7900,-152.4072,13.73,72,0,2.06,US,1722542377


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(17)

,City,Country,Lat,Lng,Humidity,Hotel Name
4,severinia,BR,-20.8094,-48.8028,20,
20,la passe,FR,45.5549,-0.8967,82,
27,saint-francois,FR,46.4154,3.9054,76,
51,udachny,RU,66.4167,112.4000,65,
73,longyearbyen,SJ,78.2186,15.6401,67,
117,arbatache,DZ,36.6377,3.3713,66,
119,katsuren-haebaru,JP,26.3369,127.8719,86,
137,mpongwe,ZM,-13.5091,28.1550,32,
174,emerald,AU,-23.5333,148.1667,55,
182,kodiak,US,57.7900,-152.4072,72,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    params["categories"] = f"accommodation.hotel"
    params["limit"]=1
    params["apiKey"]=geoapify_key

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)


    # Convert the API response to JSON format
    name_address = name_address.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(17)

Starting hotel search
severinia - nearest hotel: Hotel Nacional
la passe - nearest hotel: L'Estuaire
saint-francois - nearest hotel: Chez Lily
udachny - nearest hotel: Вилюй
longyearbyen - nearest hotel: The Vault
arbatache - nearest hotel: Hôtel Eddounia
katsuren-haebaru - nearest hotel: No hotel found
mpongwe - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
kodiak - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
inhambane - nearest hotel: Hotel de Inhambane
limulunga - nearest hotel: St. Lawrence Lodge
neon karlovasion - nearest hotel: Ξενοδοχείο Αιγαίον
knysna - nearest hotel: Graywood Hotel
nyon - nearest hotel: Hôtel Ambassador
namie - nearest hotel: HOTELなみえ
argo - nearest hotel: Apollon Hotel
birjand - nearest hotel: مهمانسرای جهانگردی بیرجند
nyurba - nearest hotel: Марина
plettenberg bay - nearest hotel: Milkwood Manor
mkushi - nearest hotel: Shalom Lodge
antalaha - nearest hotel: Hôtel Florida
nili - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
4,severinia,BR,-20.8094,-48.8028,20,Hotel Nacional
20,la passe,FR,45.5549,-0.8967,82,L'Estuaire
27,saint-francois,FR,46.4154,3.9054,76,Chez Lily
51,udachny,RU,66.4167,112.4000,65,Вилюй
73,longyearbyen,SJ,78.2186,15.6401,67,The Vault
117,arbatache,DZ,36.6377,3.3713,66,Hôtel Eddounia
119,katsuren-haebaru,JP,26.3369,127.8719,86,No hotel found
137,mpongwe,ZM,-13.5091,28.1550,32,No hotel found
174,emerald,AU,-23.5333,148.1667,55,Emerald Central Hotel
182,kodiak,US,57.7900,-152.4072,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display
hotel_data = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='OSM',
                               size='Humidity', color='City', alpha=0.6, scale=1, frame_height=600, frame_width=800, hover_cols = ['Country', 'Hotel Name'])
hotel_data

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)